In [1]:
import pandas as pd
import numpy as np
import configparser
import pymysql
import requests
import re
from tqdm import tqdm
import configparser
import os

In [2]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [3]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [4]:
df = pd.read_csv('data/epoch_registration.csv')

In [5]:
df

,id,passing_year,exp_year,EXP Bucket
0,1205309,2025,1,>0 to 3
1,1326127,2024,3,>0 to 3
2,2031026,2024,2,>0 to 3
3,2293073,2024,0,Fresher
4,2305834,2025,0,Fresher
...,...,...,...,...
4271,2705411,2024,1,>0 to 3
4272,2705413,2024,1,>0 to 3
4273,2705415,2024,0,Fresher
4274,2739965,2024,0,Fresher


In [6]:
# sub_df = df[df['exp_year']>0]

In [7]:
# sub_df

In [8]:
utup = tuple(df.id.tolist())

In [9]:
# utup

In [10]:

query = f"select id from profile_info where id in {(utup)}"

mycursor.execute(query)
result = mycursor.fetchall()

In [11]:
active_users = pd.DataFrame(result,columns=['userid'])

In [12]:
active_users.drop_duplicates(inplace=True)

In [13]:
active_users

,userid
0,1205309
1,1326127
2,2031026
3,2293073
4,2305834
...,...
4271,2705411
4272,2705413
4273,2705415
4274,2739965


In [16]:
ulist = tuple(active_users['userid'].tolist())

In [18]:
from user_info import get_user_info

In [19]:
r_text = {}
for uid in tqdm(ulist):
    response = get_user_info(uid)
    try:
        r_text[uid] = response['resume'][0]
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4276/4276 [00:28<00:00, 148.47it/s]


In [20]:
import json
with open('../custom_data/mistral/epoch-full-resume-text.json','w') as f:
    json.dump(r_text,f)